### Data Wrangling File (2020-2023). The capabilities of this file are the following: 
* Read in data from the github repository in python
* Perform preliminary data cleaning steps (Converting columns to numeric, renaming '.', '-1', '-3' values to NA)
* Mapping column IDs with real column names from refrence files 


Note: This should work for 2020 - 2023 data. Issue with 2018-19 data 


Importing necessary packages 

In [71]:
#Importing necessary packages
import pandas as pd
import os 
import numpy as np
import warnings
import re
warnings.simplefilter("ignore")

Loading in all raw data & REF files and the REF files by storing them into dictionaries (HELPER FUNCTION)

In [72]:
def load_data(directory, year):
    """
    Reads all CSV files in the specified directory into a dictionary (RAWDATA) 
    and loads all sheets from an Excel file into another dictionary (REF).

    
    
    Args:
        directory (str): The path to the directory containing the raw data files.
        year (int or str): The year to append to dictionary keys.
    
    Returns:
        tuple: (RAWDATA, REF)
        RAWDATA: Dictionary containing all raw data files 
        REF: Dictionary containing column IDs for each 
    """
    
    # Change to the specified directory
    os.chdir(directory)

    # Store all raw CSV data in a dictionary with dynamic year
    csv_files = [f for f in os.listdir() if f.endswith('.csv')]
    RAWDATA = {file.split('.')[0].lower(): pd.read_csv(file) for file in csv_files}

    # Look for an Excel file (assuming there's only one Excel file in the directory)
    excel_files = [f for f in os.listdir() if f.endswith('.xlsx') or f.endswith('.xls')]
    
    REF = {}
    if excel_files:
        file_path = excel_files[0]  # Taking the first Excel file found
        sheetname_ref = pd.ExcelFile(file_path).sheet_names
        REF = {f"{sheet}_ref{year}": pd.read_excel(file_path, sheet_name=sheet) for sheet in sheetname_ref}

    # Return both dictionaries
    return RAWDATA, REF

Primary Data Cleaning (Helper Function)
* Converts all values (except district ID) to numeric  values
* Remove NA values which are encoded as either '.', -1, or -3
* Remove values that are supposed to be percent or rates that are > 100 

In [73]:
def primary_data_cleaning(df_dict, level):
    """
    Converts all columns in each DataFrame (except the one containing the specified level) to numeric.
    Replaces '.', '-1', and '-3' values with NaN.
    Drops columns containing 'rate' or 'percent' in their name if any values exceed 100,
    except for DataFrames with 'ref' in their title.
    
    Additionally, searches the DataFrame's columns for a column that contains the specified level's long form
    and renames it to '{original_name}_id' if found.
    
    Args:
        df_dict (dict): Dictionary of pandas DataFrames.
        level (str): Level of granularity ('C', 'D', 'R', 'S') corresponding to Campus, District, Region, State.
    
    Returns:
        dict: Dictionary of DataFrames with processed data.
    """
    level_map = {
        "C": ["Campus", "District"],  # Campus level should include both Campus and District columns
        "D": ["District"],
        "R": ["Region"],
        "S": ["State"]
    }
    
    level_names = level_map.get(level)
    if not level_names:
        raise ValueError("Invalid level input. Must be one of 'C', 'D', 'R', 'S'.")
    
    processed_dict = {}
    
    for key, df in df_dict.items():
        # Skip processing for DataFrames whose key contains 'ref' or 'type' because they are different files 
        if 'ref' in key.lower() or 'type' in key.lower():
            processed_dict[key] = df.copy()
            continue
        
        df = df.copy()
        
        # Identify columns that match the specified level names
        matching_columns = [col for col in df.columns if any(name.lower() in col.lower() for name in level_names)]
        
        # Rename matching columns by appending '_id'
        for col in matching_columns:
            df.rename(columns={col: f"{col}_id"}, inplace=True)
        
        # Convert all columns except the identified level columns to numeric
        id_columns = [f"{col}_id" for col in matching_columns]
        for col in df.columns:
            if col not in id_columns:
                df[col] = df[col].replace({'.': np.nan, '-1': np.nan, '-3': np.nan})  # Replace invalid values with NaN
                df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric
            else:
                df[col] = df[col].astype(str)  # Ensure ID columns remain as strings
        
        processed_dict[key] = df
    
    return processed_dict



Mapping the column ID names in the raw data with the actual column names in the REF file (HELPER FUNCTION)

In [74]:
def rename_columns_using_ref(rawdata, ref):
    """
    Renames columns in each DataFrame in rawdata using the corresponding mapping found in ref.
    If a filename contains 'ref' or 'type', it is copied unchanged.

    Args:
        rawdata (dict): Dictionary containing raw DataFrames with keys as filenames.
        ref (dict): Dictionary containing reference DataFrames with keys as filenames.

    Returns:
        dict: Dictionary containing renamed DataFrames.
    """
    
    updated_data = {}  # Dictionary to store updated DataFrames

    for raw_key, raw_df in rawdata.items():
        # Skip processing if key contains 'ref' or 'type'
        if 'ref' in raw_key.lower() or 'type' in raw_key.lower():
            updated_data[raw_key] = raw_df.copy()
            continue
        
        # Extract base name before the first underscore (_)
        base_name = raw_key.split("_")[0]
        
        # Find the matching key in REF (case-insensitive)
        matching_key = next((key for key in ref if key.lower().startswith(base_name.lower())), None)
        
        if matching_key:
            # Extract mapping from REF (second column = column ID, third column = actual column name)
            ref_df = ref[matching_key]
            column_mapping = dict(zip(ref_df.iloc[:, 1], ref_df.iloc[:, 2]))  # Map column ID → Actual name
            
            # Rename columns in RAWDATA DataFrame
            renamed_df = raw_df.rename(columns=column_mapping)
        else:
            # If no match is found, keep the DataFrame unchanged
            renamed_df = raw_df.copy()

        # Store in updated_data with the original key
        updated_data[raw_key] = renamed_df

    # Print confirmation
    print(f"Processed {len(updated_data)} DataFrames (Renamed: {len([k for k in updated_data if k not in rawdata or ('ref' not in k.lower() and 'type' not in k.lower())])}, Unchanged: {len([k for k in updated_data if 'ref' in k.lower() or 'type' in k.lower()])}).")

    return updated_data


In [ ]:
def join_with_reference(df_dict, level):
    """
    Identifies the DataFrame with 'ref' in its key and left joins it with all other DataFrames 
    (except those containing '_type' in their key). The join is performed using {LEVEL} in the 
    reference DataFrame and f"{LEVEL}_id" in the other DataFrames.

    Ensures that the merged columns are converted to strings before merging.

    Args:
        df_dict (dict): Dictionary of pandas DataFrames.
        level (str): Level of granularity ('C', 'D', 'R', 'S') corresponding to Campus, District, Region, State.

    Returns:
        dict: Updated dictionary with joined data.
    """
    level_full_name = {
        "C": "CAMPUS",  
        "D": "DISTRICT",
        "R": "REGION",
        "S": "STATE"
    }[level]

    ref_df = None
    ref_key = None

    # Locate the reference DataFrame
    for key in df_dict.keys():
        if 'ref' in key.lower():
            ref_df = df_dict[key]
            ref_key = key
            break  # Only one reference DataFrame is assumed

    if ref_df is None:
        raise ValueError("No reference DataFrame found in the dictionary keys.")

    join_col_ref = level_full_name  # Column name in the reference DataFrame
    join_col_main = f"{level_full_name}_id"  # Column name in other DataFrames

    print(f"Joining on: ref[{join_col_ref}] with main[{join_col_main}]")

    # Convert the reference column to string
    ref_df[join_col_ref] = ref_df[join_col_ref].astype(str)

    updated_dict = {}

    for key, df in df_dict.items():
        if key == ref_key or '_type' in key.lower():  
            # Keep the reference and '_type' DataFrames unchanged
            updated_dict[key] = df
        elif join_col_main in df.columns:  
            # Convert the main DataFrame's join column to string before merging
            df[join_col_main] = df[join_col_main].astype(str)

            # Perform the left join
            updated_dict[key] = df.merge(ref_df, how='left', left_on=join_col_main, right_on=join_col_ref)
        else:
            # If no matching column, keep the DataFrame unchanged
            updated_dict[key] = df

    return updated_dict


In [89]:
def processing(directory, year, level):
    """
    Processes raw data by loading, cleaning, renaming columns, and joining level refrence.

    Args:
        directory (str): Path to the directory containing the data.
        year (int): Year of the data to be processed.
        level (str): Cleaning level or category for data processing.

    Returns:
        DataFrame: The cleaned and processed data with renamed columns.
    """
    rawdata, ref = load_data(directory, year)
    cleaned_data = primary_data_cleaning(rawdata, level)
    column_data = rename_columns_using_ref(cleaned_data, ref)
    if level != 'S':
        final_data = join_with_reference(column_data, level)
        return final_data
    else:
        return column_data


In [27]:
def process_and_save_all_data(base_directory, level):
    """
    Loops through all Data{year} folders, processes the data, and saves the output
    as multiple Excel files in the corresponding clean_data folder within each level.
    
    Parameters:
    base_directory (str): Path to the folder containing Data{year} folders.
    level (str): Level parameter required for data processing.
    """
    # Get the full level name 
    valid_levels = {
        'C': 'Campus',
        'D': 'District',
        'R': 'Region',
        'S': 'State'
    }

    # Get all folder names and extract years
    year_folders = [f for f in os.listdir(base_directory) if f.startswith('Data')]
    years = sorted([int(f.replace('Data', '')) for f in year_folders if f.replace('Data', '').isdigit() and int(f.replace('Data', '')) >= 2020])
    
    for year in years:
        data_year_folder = os.path.join(base_directory, f'Data{year}')
        raw_data_folder = os.path.join(data_year_folder, f'{valid_levels[level]}', 'raw_data')
        clean_data_folder = os.path.join(data_year_folder, f'{valid_levels[level]}', 'clean_data')
        os.makedirs(clean_data_folder, exist_ok=True)
        
        if os.path.exists(raw_data_folder):
            # Check if all output files already exist before processing
            processed_data_needed = False
            
            for file_name in os.listdir(raw_data_folder):
                clean_file_name = f"{os.path.splitext(file_name)[0]}_clean.xlsx"
                output_file = os.path.join(clean_data_folder, clean_file_name)
                if not os.path.exists(output_file):
                    processed_data_needed = True
                    break
            
            if not processed_data_needed:
                print(f"Skipping processing for year {year} at level {level} as all files already exist.")
                continue
            
            print(f"Processing data for year {year} at level {level}...")
            
            # Run the processing function, which returns a dictionary of DataFrames
            processed_data = processing(raw_data_folder, year, level)
            
            # Save each DataFrame in the dictionary as a separate Excel file
            for file_name, df in processed_data.items():
                clean_file_name = f"{file_name}_clean.xlsx"
                output_file = os.path.join(clean_data_folder, clean_file_name)
                
                if os.path.exists(output_file):
                    print(f"Skipping {clean_file_name} as it already exists.")
                else:
                    df.to_excel(output_file, index=False)
                    print(f"Saved cleaned data for {year}, level {level}: {clean_file_name}")
        else:
            print(f"Warning: Raw data folder for {year} at level {level} does not exist, skipping...")


In [28]:
# Example usage
base_dir = r"C:\Users\mmath\OneDrive\Desktop\Capstone\HERC_Sp25\Data"
process_and_save_all_data(base_dir, 'S')


Processing data for year 2020 at level S...
Processed 4 DataFrames (Renamed: 4, Unchanged: 0).
Skipping stategrad_2020_clean.xlsx as it already exists.
Skipping stateperf_2020_clean.xlsx as it already exists.
Skipping stateprof_2020_clean.xlsx as it already exists.
Skipping statestaar1_2020_clean.xlsx as it already exists.
Processing data for year 2021 at level S...
Processed 4 DataFrames (Renamed: 4, Unchanged: 0).
Skipping stategrad_2021_clean.xlsx as it already exists.
Skipping stateperf1_2021_clean.xlsx as it already exists.
Skipping stateprof_2021_clean.xlsx as it already exists.
Skipping statestaar1_2021_clean.xlsx as it already exists.
Processing data for year 2022 at level S...
Processed 4 DataFrames (Renamed: 4, Unchanged: 0).
Skipping stategrad_2022_clean.xlsx as it already exists.
Skipping stateperf1_2022_clean.xlsx as it already exists.
Skipping stateprof_2022_clean.xlsx as it already exists.
Skipping statestaar1_2022_clean.xlsx as it already exists.
Processing data for yea

In [33]:
def merge_cleaned_data(base_directory, level):
    """
    Merges all cleaned Excel files from different years at the specified level and joins with the corresponding yearly reference file.

    Parameters:
    - base_directory (str): Path to the folder containing Data{year} folders.
    - level (str): Level parameter (C, D, R, S).

    Returns:
    - Merged DataFrame
    """
    valid_levels = {'C': 'Campus', 'D': 'District', 'R': 'Region', 'S': 'State'}
    level_name = valid_levels[level]  # Example: 'District'
    id_column = f"{level_name.lower()}_id"  # Example: 'district_id'
    
    all_data = []
    
    # Get year folders
    year_folders = [f for f in os.listdir(base_directory) if f.startswith('Data')]
    years = sorted([int(f.replace('Data', '')) for f in year_folders if f.replace('Data', '').isdigit() and int(f.replace('Data', '')) >= 2020])
    
    for year in years:
        clean_data_folder = os.path.join(base_directory, f'Data{year}', level_name, 'clean_data')

        if not os.path.exists(clean_data_folder):
            print(f"Skipping {year}, clean data folder does not exist.")
            continue
        
        # Locate reference file for this year
        ref_file_name = f"{level.lower()}ref_{year}_clean.xlsx"
        ref_file_path = os.path.join(base_directory, ref_file_name)
        
        if os.path.exists(ref_file_path):
            ref_df = pd.read_excel(ref_file_path)
            print(f"Found reference file for {year}: {ref_file_path}")
        else:
            print(f"Warning: Reference file '{ref_file_name}' not found for {year}. Skipping reference merge for this year.")
            ref_df = None

        # Load and merge all Excel files in the clean_data_folder
        year_data = []
        for file_name in os.listdir(clean_data_folder):
            if file_name.endswith("_clean.xlsx"):
                file_path = os.path.join(clean_data_folder, file_name)
                df = pd.read_excel(file_path)
                df['year'] = year  # Add a year column for tracking
                year_data.append(df)
        
        if year_data:
            merged_year_df = pd.concat(year_data, ignore_index=True)

            # Merge with reference file if available
            if ref_df is not None and id_column in ref_df.columns:
                merged_year_df = merged_year_df.merge(ref_df, on=id_column, how="left")
                print(f"Successfully merged {year} data with reference file on {id_column}.")
            elif ref_df is not None:
                print(f"Warning: Reference file for {year} found but missing '{id_column}', skipping reference merge.")

            all_data.append(merged_year_df)
    
    if not all_data:
        print("No data found to merge.")
        return None
    
    # Combine all years into one DataFrame
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df


In [ ]:
#base_dir = r"C:\Users\mmath\OneDrive\Desktop\Capstone\HERC_Sp25\Data"


pd.read_excel(r'C:\Users\mmath\OneDrive\Desktop\Capstone\HERC_Sp25\Data\Data2020\District\clean_data\distperf_2020_clean.xlsx')

,DISTRICT_id,"District 2019 College, Career, & Military Ready Graduates: All Students Numerator","District 2019 College, Career, & Military Ready Graduates: All Students Denominator","District 2019 College, Career, & Military Ready Graduates: African American Numerator","District 2019 College, Career, & Military Ready Graduates: African American Denominator","District 2019 College, Career, & Military Ready Graduates: Hispanic Numerator","District 2019 College, Career, & Military Ready Graduates: Hispanic Denominator","District 2019 College, Career, & Military Ready Graduates: White Numerator","District 2019 College, Career, & Military Ready Graduates: White Denominator","District 2019 College, Career, & Military Ready Graduates: American Indian Numerator",...,"District TX IHE 2018: % of 2017 White Graduates Enrolled in TX Public Higher Ed in School Year 2018, Who Met TSI in All Subjects",District TX IHE 2018: % of 2017 White Graduates Enrolled in TX Public or Independent Higher Ed in School Year 2018,"District TX IHE 2018: % of 2017 Asian Graduates Enrolled in TX Public Higher Ed in School Year 2018, Who Met TSI in All Subjects",District TX IHE 2018: % of 2017 Asian Graduates Enrolled in TX Public or Independent Higher Ed in School Year 2018,"District TX IHE 2018: % of 2017 American Indian Graduates Enrolled in TX Public Higher Ed in School Year 2018, Who Met TSI in All Subjects",District TX IHE 2018: % of 2017 American Indian Graduates Enrolled in TX Public or Independent Higher Ed in School Year 2018,"District TX IHE 2018: % of 2017 Female Graduates Enrolled in TX Public Higher Ed in School Year 2018, Who Met TSI in All Subjects",District TX IHE 2018: % of 2017 Female Graduates Enrolled in TX Public or Independent Higher Ed in School Year 2018,"District TX IHE 2018: % of 2017 LEP Graduates Enrolled in TX Public Higher Ed in School Year 2018, Who Met TSI in All Subjects",District TX IHE 2018: % of 2017 LEP Graduates Enrolled in TX Public or Independent Higher Ed in School Year 2018
0,1902,41.0,48.0,NaN,NaN,NaN,NaN,37.5,43.0,NaN,...,45.8,56.8,NaN,NaN,NaN,NaN,64.3,75.0,NaN,NaN
1,1903,66.0,80.0,NaN,NaN,4.5,7.0,58.5,69.0,NaN,...,70.4,39.1,NaN,NaN,NaN,NaN,65.0,47.6,NaN,NaN
2,1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.6,41.9,NaN,NaN,NaN,NaN,85.7,29.2,NaN,NaN
3,1906,19.0,22.0,NaN,NaN,NaN,NaN,14.0,17.0,NaN,...,90.0,83.3,NaN,NaN,NaN,NaN,85.7,63.6,NaN,NaN
4,1907,180.5,207.0,51.5,63.0,76.0,84.0,46.5,52.0,NaN,...,78.6,39.2,NaN,NaN,NaN,NaN,64.5,55.2,NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,252902,13.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44.4,55.6,NaN,NaN,NaN,NaN,50.0,90.0,NaN,NaN
1198,252903,31.0,44.0,NaN,NaN,12.5,18.0,18.0,24.0,NaN,...,68.4,57.6,NaN,NaN,NaN,NaN,61.5,65.0,NaN,NaN
1199,253901,177.5,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62.5,55.6,NaN,14.3
1200,254901,77.5,118.0,NaN,NaN,76.0,116.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,28.6,60.3,NaN,37.5


In [40]:
pd.read_excel(r'C:\Users\mmath\OneDrive\Desktop\Capstone\HERC_Sp25\Data\Data2020\District\clean_data\dref_2020_clean.xlsx')

,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,DFLCHART,DFLALTED,D_RATING,OUTCOME,ASVAB_STATUS
0,1902,CAYUGA ISD,1,ANDERSON,7,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
1,1903,ELKHART ISD,1,ANDERSON,7,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
2,1904,FRANKSTON ISD,1,ANDERSON,7,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
3,1906,NECHES ISD,1,ANDERSON,7,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
4,1907,PALESTINE ISD,1,ANDERSON,7,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
...,...,...,...,...,...,...,...,...,...,...
1197,252902,NEWCASTLE ISD,252,YOUNG,9,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
1198,252903,OLNEY ISD,252,YOUNG,9,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
1199,253901,ZAPATA COUNTY ISD,253,ZAPATA,1,N,N,Not Rated: Declared State of Disaster,Meets Requirements,
1200,254901,CRYSTAL CITY ISD,254,ZAVALA,20,N,N,Not Rated: Declared State of Disaster,Needs Assistance,


Master data wrangling function (MAIN FUNCTION)
* Reads in data
* Maps column ids with real column names 
* Performs primary data cleaning

In [ ]:
import pandas as pd
import numpy as np

def primary_data_cleaning(df_dict, level):
    """
    Cleans the data by converting columns to numeric, renaming level-specific columns,
    and handling missing values.
    
    Args:
        df_dict (dict): Dictionary of pandas DataFrames.
        level (str): Level of granularity ('C', 'D', 'R', 'S') corresponding to Campus, District, Region, State.
    
    Returns:
        tuple: Processed dictionary of DataFrames and the reference DataFrame (if found).
    """
    level_map = {
        "C": ["Campus", "District"],  # Campus level should include both Campus and District columns
        "D": ["District"],
        "R": ["Region"],
        "S": ["State"]
    }
    
    level_names = level_map.get(level)
    if not level_names:
        raise ValueError("Invalid level input. Must be one of 'C', 'D', 'R', 'S'.")

    processed_dict = {}
    ref_df = None
    ref_key = None

    for key, df in df_dict.items():
        df = df.copy()

        # Identify reference DataFrame (contains 'ref' in the key)
        if 'ref' in key.lower():
            ref_df = df
            ref_key = key
            processed_dict[key] = df  # Store the reference DataFrame unchanged
            continue

        # Identify columns that match the specified level names
        matching_columns = [col for col in df.columns if any(name.lower() in col.lower() for name in level_names)]

        # Rename matching columns by appending '_id'
        for col in matching_columns:
            df.rename(columns={col: f"{col}_id"}, inplace=True)

        # Convert all columns except the identified level columns to numeric
        id_columns = [f"{col}_id" for col in matching_columns]
        for col in df.columns:
            if col not in id_columns:
                df[col] = df[col].replace({'.': np.nan, '-1': np.nan, '-3': np.nan})  # Replace invalid values with NaN
                df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric
            else:
                df[col] = df[col].astype(str)  # Ensure ID columns remain as strings

        processed_dict[key] = df

    return processed_dict, ref_df  # Return both processed data and reference DataFrame


def merge_reference_data(processed_dict, ref_df, level):
    """
    Merges the reference DataFrame with all other DataFrames in processed_dict based on the level.

    Args:
        processed_dict (dict): Dictionary of cleaned pandas DataFrames.
        ref_df (pd.DataFrame): Reference DataFrame containing additional information.
        level (str): Level of granularity ('C', 'D', 'R', 'S') corresponding to Campus, District, Region, State.

    Returns:
        dict: Updated dictionary with reference data merged.
    """
    if ref_df is None:
        return processed_dict  # No reference data found, return original dictionary

    # Mapping levels to expected ID column names
    level_join_col_map = {
        "C": "CAMPUS_id",
        "D": "DISTRICT_id",
        "R": "REGION_id",
        "S": "STATE_id"
    }

    ref_join_col_map = {
        "C": "CAMPUS",
        "D": "DISTRICT",
        "R": "REGION",
        "S": "STATE"
    }

    level_join_col = level_join_col_map.get(level)
    ref_join_col = ref_join_col_map.get(level)

    if not level_join_col or not ref_join_col:
        raise ValueError("Invalid level input. Must be one of 'C', 'D', 'R', 'S'.")

    for key in list(processed_dict.keys()):
        if level_join_col in processed_dict[key].columns:
            processed_dict[key] = processed_dict[key].merge(ref_df, how='left', left_on=level_join_col, right_on=ref_join_col)

    return processed_dict

